<a href="https://colab.research.google.com/github/nfamartins/reconhecimento-padroes/blob/main/pre_processamento_lattes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparando o ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Definindo um diretório base

In [2]:
import os
base_dir = '/content/drive/MyDrive/Projetos/each/sin5007'
os.chdir(base_dir)

Importando bibliotecas

In [3]:
import pandas as pd

# Importando dados

## Dicionários das tabelas

Os dados estão disponibilizados em diferentes tabelas (sem os nomes das colunas). Vamos baixar o dicionário das tabelas e variáveis.

In [6]:
dic = pd.read_csv('dados/dicionario_tabelas.csv', sep = '\t')

Visualizando as tabelas disponíves:

In [7]:
dic['nome_tabela'].unique()

array(['areas', 'atuacoes', 'enderecos', 'formacoes', 'gerais',
       'periodicos', 'orientacoes'], dtype=object)

## Dados gerais dos doutores no Lattes

Dados gerais, como nome e local de nascimento.

In [13]:
# dados gerais dos doutores
# vou ler todos os campos como object, caso quisesse especificar, basta passar um dicionário
docs = pd.read_csv('dados/gerais.csv', sep='\t', header = None, dtype = 'object')

In [14]:
#verificando
docs.head()

,0,1,2,3,4,5,6,7,8
0,0000039015885890,Tárcio de Almeida Vieira,29042021,123614,Ipiau,BA,Brasil,B,NaN
1,0000104116232000,Andre Zuber,31082018,134523,Irati,PR,Brasil,B,NaN
2,0000111305956227,Tecia Vieira Carvalho,24092021,163956,Parnaiba,PI,Brasil,B,NaN
3,0000174385402367,Juliana Vieira de Freitas,04062021,111421,Sorocaba,SP,Brasil,B,NaN
4,0000183606503103,Luiza Cristina Lacerda,25112014,004523,Goliânia,GO,Brasil,B,NaN


In [15]:
# a tabela está vindo com uma coluna a mais (vazia) do que o esperado, então vamos dropa-la
docs.drop(8, axis=1, inplace=True)

# e renomear com os col_names do dicionario
docs.columns = dic[dic['nome_tabela'] == 'gerais']['col_names'].values

In [16]:
# verificando
docs.head()

,id_lattes,nome_completo,data_update,hora_update,cidade_nascimento,uf_nascimento,pais_nascimento,nacionalidade
0,0000039015885890,Tárcio de Almeida Vieira,29042021,123614,Ipiau,BA,Brasil,B
1,0000104116232000,Andre Zuber,31082018,134523,Irati,PR,Brasil,B
2,0000111305956227,Tecia Vieira Carvalho,24092021,163956,Parnaiba,PI,Brasil,B
3,0000174385402367,Juliana Vieira de Freitas,04062021,111421,Sorocaba,SP,Brasil,B
4,0000183606503103,Luiza Cristina Lacerda,25112014,004523,Goliânia,GO,Brasil,B


In [17]:
print(len(docs))
print(docs['id_lattes'].nunique())

373590
373590


Ok. Banco de dados dos locais de nascimento, únicos para cada id_lattes do banco

## Áreas de atuação dos doutores

In [18]:
# dados gerais dos doutores
# vou ler todos os campos como object, caso quisesse especificar, basta passar um dicionário
area = pd.read_csv('dados/areas.csv', sep='\t', header = None, dtype='object')

In [20]:
# renomear com os col_names do dicionario
area.columns = dic[dic['nome_tabela'] == 'areas']['col_names'].values

In [21]:
#verificando
area.head()

,id_lattes,area_atuacao_seq,grande_area,area_conhecimento,subarea_conhecimento,especialidade
0,0000039015885890,3,CIENCIAS_EXATAS_E_DA_TERRA,Física,"Dispositivos Ópticos, Optoeletrônicos e Fotônicos",NaN
1,0000039015885890,2,CIENCIAS_EXATAS_E_DA_TERRA,Física,Ótica não linear Aplicada,NaN
2,0000039015885890,1,CIENCIAS_EXATAS_E_DA_TERRA,Física,Materiais Eletro-Ópticos e Fotorrefrativos,NaN
3,0000104116232000,1,ENGENHARIAS,Engenharia Química,NaN,NaN
4,0000111305956227,6,CIENCIAS_SOCIAIS_APLICADAS,Administração,Administração de Setores Específicos,NaN


In [22]:
print(len(area))
print(area['id_lattes'].nunique())

1256369
344540


Os usuários do Lattes podem selecionar mais de uma área de atuação. Para simplificação, no momento, selecionaremos apenas a área de atuação principal (area_atuacao == 1)

In [23]:
area = area[area['area_atuacao_seq'] == '1']
print(len(area))
print(area['id_lattes'].nunique())

343065
343019


Ainda existem dados repetivos na tabela, mas por enquanto, vamos ignorar o erro e eliminar os os ids repetidos

In [24]:
area = area.drop_duplicates(subset='id_lattes', keep='first')
print(len(area))

343019


Adicionando esses dados ao df principal (doc)

In [25]:
docs = pd.merge(docs, area, on='id_lattes', how='left')

In [26]:
docs.head()

,id_lattes,nome_completo,data_update,hora_update,cidade_nascimento,uf_nascimento,pais_nascimento,nacionalidade,area_atuacao_seq,grande_area,area_conhecimento,subarea_conhecimento,especialidade
0,0000039015885890,Tárcio de Almeida Vieira,29042021,123614,Ipiau,BA,Brasil,B,1,CIENCIAS_EXATAS_E_DA_TERRA,Física,Materiais Eletro-Ópticos e Fotorrefrativos,NaN
1,0000104116232000,Andre Zuber,31082018,134523,Irati,PR,Brasil,B,1,ENGENHARIAS,Engenharia Química,NaN,NaN
2,0000111305956227,Tecia Vieira Carvalho,24092021,163956,Parnaiba,PI,Brasil,B,1,CIENCIAS_EXATAS_E_DA_TERRA,Química,Química Orgânica,Síntese Orgânica
3,0000174385402367,Juliana Vieira de Freitas,04062021,111421,Sorocaba,SP,Brasil,B,1,ENGENHARIAS,Engenharia Química,Processos Industriais de Engenharia Química,NaN
4,0000183606503103,Luiza Cristina Lacerda,25112014,004523,Goliânia,GO,Brasil,B,1,CIENCIAS_BIOLOGICAS,Farmacologia,NaN,NaN


In [ ]:
len(docs)

373590

Neste momento, também não utilizaremos algumas variáveis. Como data e hora de atualização, cidade de nascimento e ficaremos apenas com a grande área de atuação. Então, eliminando essas variáveis.

In [28]:
docs = docs.drop(columns = ['data_update','hora_update','cidade_nascimento','area_atuacao_seq',
                            'area_conhecimento','subarea_conhecimento','especialidade'])
docs.head()

,id_lattes,nome_completo,uf_nascimento,pais_nascimento,nacionalidade,grande_area
0,0000039015885890,Tárcio de Almeida Vieira,BA,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA
1,0000104116232000,Andre Zuber,PR,Brasil,B,ENGENHARIAS
2,0000111305956227,Tecia Vieira Carvalho,PI,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA
3,0000174385402367,Juliana Vieira de Freitas,SP,Brasil,B,ENGENHARIAS
4,0000183606503103,Luiza Cristina Lacerda,GO,Brasil,B,CIENCIAS_BIOLOGICAS


Apagando o df de áreas

In [30]:
del area

## Importando dados de papers (periódicos)

In [29]:
# o arquivo está com problemas de padronização dos dados
# usaremos error_bad_lines=False para conseguir ler os dados, e trataremos esses problemas em outro momento
papers = pd.read_csv('dados/periodicos.csv', sep='\t', header = None, on_bad_lines='warn', dtype = 'object',
                     names=dic[dic['nome_tabela'] == 'periodicos']['col_names'].values)

In [32]:
papers.head(3)

,id_lattes,ano_artigo,titulo_artigo,natureza,idioma,issn,tituto_periodico,pagina_inicial,pagina_final,volume,fasciculo,serie,local_publicacao,pais_publicacao,doi,grande_area,area_conhecimento,subarea_conhecimento,especialidade
0,0041178224320635,2020,Facial profile changes due to bone cement graf...,COMPLETO,Inglês,21776709,DENTAL PRESS JOURNAL OF ORTHODONTICS,44,55,25,NaN,NaN,NaN,NaN,10.1590/2177-6709.25.2.044-051.oar,CIENCIAS_DA_SAUDE,NaN,Periodontia,Periodontia
1,0041178224320635,2005,Correlação entre Espessura Gengival nos Enxert...,COMPLETO,Português,01047914,ROBRAC (GOIÂNIA. IMPRESSO),27,31,14,NaN,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0041178224320635,2012,. Enxerto de tecido conjuntivo subepitelial co...,COMPLETO,Português,24477567,ISSN 2447-7567,165,170,6,NaN,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fazendo uma análise simples de contagem de papers publicados (total geral). Depois precisaremos fazer análises mais sofisticadas, por ano, antes/após o doutorado, impacto das revistas, área de publicação, etc.

In [33]:
papers_qtd = papers.groupby('id_lattes').size().reset_index(name='n_papers')

In [34]:
papers_qtd.head()

,id_lattes,n_papers
0,0000039015885890,9
1,0000104116232000,8
2,0000111305956227,17
3,0000174385402367,5
4,0000183606503103,4


In [35]:
len(papers_qtd)

338023

Adicionando ao df principal (docs)

In [36]:
docs = pd.merge(docs, papers_qtd, on='id_lattes', how='left')

In [37]:
docs.head()

,id_lattes,nome_completo,uf_nascimento,pais_nascimento,nacionalidade,grande_area,n_papers
0,0000039015885890,Tárcio de Almeida Vieira,BA,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,9.0
1,0000104116232000,Andre Zuber,PR,Brasil,B,ENGENHARIAS,8.0
2,0000111305956227,Tecia Vieira Carvalho,PI,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,17.0
3,0000174385402367,Juliana Vieira de Freitas,SP,Brasil,B,ENGENHARIAS,5.0
4,0000183606503103,Luiza Cristina Lacerda,GO,Brasil,B,CIENCIAS_BIOLOGICAS,4.0


In [38]:
len(docs)

373590

Apagando os dfs

In [39]:
del papers, papers_qtd

## Importando dados de orientações

In [40]:
orientacoes = pd.read_csv('dados/orientacoes.csv', sep='\t', header = None, dtype = 'object',
                     names=dic[dic['nome_tabela'] == 'orientacoes']['col_names'].values)

In [41]:
orientacoes.head()

,id_lattes,natureza_orientacao,status_orientacao,ano_orientacao,nome_orientador,id_instituicao,nome_instituicao,id_curso,bolsa_flag,id_agencia_financiadora,nome_agencia_financiadora,titulo_orientacao,id_orientado,nome_curso,grande_area,area_conhecimento,subarea_conhecimento,especialidade,tipo_orientacao
0,0041178224320635,Dissertação de mestrado,CONCLUIDA,1996,Sérgio Loiola,010600000009,Universidade Federal de Goiás,90000007,NAO,NaN,NaN,Pesquisa sobre adesão das resinas na porcelana,NaN,Odontologia Prótese Dental,CIENCIAS_DA_SAUDE,NaN,Clínica Odontológica,NaN,ORIENTADOR_PRINCIPAL
1,0041178224320635,Dissertação de mestrado,CONCLUIDA,1996,Raulino Naves Borges,010600000009,Universidade Federal de Goiás,90000007,NAO,NaN,NaN,Relação entre desarmonia oclusal e cefaléias,NaN,Odontologia Prótese Dental,CIENCIAS_DA_SAUDE,NaN,Clínica Odontológica,NaN,ORIENTADOR_PRINCIPAL
2,0041178224320635,Tese de doutorado,CONCLUIDA,2013,Germana Jayme Borges,010600000009,Universidade Federal de Goiás,60026987,SIM,045000000000,Coordenação de Aperfeiçoamento de Pessoal de N...,Avaliação da tomografia computadorizada de fei...,NaN,Ciências da Saúde,CIENCIAS_DA_SAUDE,NaN,Periodontia,NaN,CO_ORIENTADOR
3,0041178224320635,MONOGRAFIA_DE_CONCLUSAO_DE_CURSO_APERFEICOAMEN...,CONCLUIDA,1996,Sirlene Rossi Mendonça,010600000009,Universidade Federal de Goiás,90000014,NAO,NaN,NaN,Limites da indicação das próteses sob o ponto ...,NaN,Prótese Dentária,CIENCIAS_DA_SAUDE,NaN,Clínica Odontológica,ORIENTADOR_PRINCIPAL,NaN
4,0041178224320635,MONOGRAFIA_DE_CONCLUSAO_DE_CURSO_APERFEICOAMEN...,CONCLUIDA,1996,Djalma Ribeiro Neves,010600000009,Universidade Federal de Goiás,90000014,NAO,NaN,NaN,Tempo de cicatrização para os preparos após in...,NaN,Prótese Dentária,CIENCIAS_DA_SAUDE,NaN,Clínica Odontológica,ORIENTADOR_PRINCIPAL,NaN


In [42]:
orientacoes['natureza_orientacao'].unique()

array(['Dissertação de mestrado', 'Tese de doutorado',
       'MONOGRAFIA_DE_CONCLUSAO_DE_CURSO_APERFEICOAMENTO_E_ESPECIALIZACAO',
       'TRABALHO_DE_CONCLUSAO_DE_CURSO_GRADUACAO', 'INICIACAO_CIENTIFICA',
       'Monografia de conclusão de curso de aperfeiçoamento/especialização',
       'Trabalho de conclusão de curso de graduação',
       'Iniciação Científica', 'ORIENTACAO-DE-OUTRA-NATUREZA',
       'Supervisão de pós-doutorado', 'Orientação de outra natureza',
       'Disserta&amp;ccedil;&amp;atilde;o de mestrado',
       'Monografia de conclus&amp;atilde;o de curso de aperfei&amp;ccedil;oamento/especializa&amp;ccedil;&amp;atilde;o',
       'Trabalho de conclus&amp;atilde;o de curso de gradua&amp;ccedil;&amp;atilde;o',
       'Inicia&amp;ccedil;&amp;atilde;o Cient&amp;iacute;fica',
       'Disserta??o de mestrado', 'Inicia??o Cient?fica'], dtype=object)

Neste momento, devido à não padronização do campo natureza_orientacao, vamos usar apenas uma contagem total de orientações.

In [43]:
orientacoes_qtd = orientacoes.groupby('id_lattes').size().reset_index(name='n_orientacoes')

In [44]:
orientacoes_qtd.head()

,id_lattes,n_orientacoes
0,0000104116232000,9
1,0000111305956227,41
2,0000183606503103,9
3,0000221437884487,27
4,0000230621439945,29


In [45]:
len(orientacoes_qtd)

268561

Adicionando ao df proncipal (docs)

In [46]:
docs = pd.merge(docs, orientacoes_qtd, on='id_lattes', how='left')
docs.head()

,id_lattes,nome_completo,uf_nascimento,pais_nascimento,nacionalidade,grande_area,n_papers,n_orientacoes
0,0000039015885890,Tárcio de Almeida Vieira,BA,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,9.0,NaN
1,0000104116232000,Andre Zuber,PR,Brasil,B,ENGENHARIAS,8.0,9.0
2,0000111305956227,Tecia Vieira Carvalho,PI,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,17.0,41.0
3,0000174385402367,Juliana Vieira de Freitas,SP,Brasil,B,ENGENHARIAS,5.0,NaN
4,0000183606503103,Luiza Cristina Lacerda,GO,Brasil,B,CIENCIAS_BIOLOGICAS,4.0,9.0


Apagando variáveis

In [47]:
del orientacoes, orientacoes_qtd

## Importando dados de formação

In [48]:
formacao = pd.read_csv('dados/formacoes.csv', sep='\t', header = None, dtype = 'object',
                     names=dic[dic['nome_tabela'] == 'formacoes']['col_names'].values)

In [49]:
formacao.head()

,id_lattes,formacao_seq,nivel_formacao,nivel,ano_inicio,ano_fim,status_curso,nome_curso,nome_instituicao,bolsa_flag
0,0000039015885890,10,POS-DOUTORADO,5,2017,NaN,EM_ANDAMENTO,NaN,"Instituto de Astronomia, Geofísica e Ciências ...",SIM
1,0000039015885890,8,POS-DOUTORADO,5,2015,2017,CONCLUIDO,NaN,Instituto de Pesquisas Energeticas e Nucleares,SIM
2,0000039015885890,7,POS-DOUTORADO,5,2014,2015,CONCLUIDO,NaN,"Instituto de Física, USP - São Carlos",SIM
3,0000039015885890,9,POS-DOUTORADO,5,2016,2017,CONCLUIDO,NaN,Universidade Federal do ABC,SIM
4,0000039015885890,3,DOUTORADO,4,2010,2014,CONCLUIDO,Física,Universidade Federal do ABC,SIM


Tipos de fomamação do banco

In [50]:
formacao['nivel_formacao'].unique()

array(['POS-DOUTORADO', 'DOUTORADO', 'MESTRADO', 'GRADUACAO',
       'ESPECIALIZACAO', 'APERFEICOAMENTO',
       'CURSO-TECNICO-PROFISSIONALIZANTE', 'ENSINO-MEDIO-SEGUNDO-GRAU',
       'ENSINO-FUNDAMENTAL-PRIMEIRO-GRAU', 'MESTRADO-PROFISSIONALIZANTE',
       'RESIDENCIA-MEDICA', 'LIVRE-DOCENCIA'], dtype=object)

Neste momento, vamos apenas analisar a quantidade de pós-doc. Filtrando para apenas pós-docs

In [51]:
formacao = formacao[formacao['nivel_formacao'] == 'POS-DOUTORADO']
print(len(formacao))
print(formacao['id_lattes'].nunique())

155461
109371


In [52]:
formacao_qtd = formacao.groupby('id_lattes').size().reset_index(name='n_posdoc')

In [53]:
formacao_qtd.head()

,id_lattes,n_posdoc
0,0000039015885890,4
1,0000104116232000,1
2,0000111305956227,1
3,0000254371229748,1
4,0000325690951570,1


Adicionando ao df principal (docs)

In [54]:
docs = pd.merge(docs, formacao_qtd, on='id_lattes', how='left')

In [55]:
docs.head()

,id_lattes,nome_completo,uf_nascimento,pais_nascimento,nacionalidade,grande_area,n_papers,n_orientacoes,n_posdoc
0,0000039015885890,Tárcio de Almeida Vieira,BA,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,9.0,NaN,4.0
1,0000104116232000,Andre Zuber,PR,Brasil,B,ENGENHARIAS,8.0,9.0,1.0
2,0000111305956227,Tecia Vieira Carvalho,PI,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,17.0,41.0,1.0
3,0000174385402367,Juliana Vieira de Freitas,SP,Brasil,B,ENGENHARIAS,5.0,NaN,NaN
4,0000183606503103,Luiza Cristina Lacerda,GO,Brasil,B,CIENCIAS_BIOLOGICAS,4.0,9.0,NaN


Apagando variáveis

In [56]:
del formacao, formacao_qtd

## Importando dados de atuação profissional

In [63]:
atuacao = pd.read_csv('dados/atuacoes.csv', sep='\t', header = None, dtype = 'object',
                     names=dic[dic['nome_tabela'] == 'atuacoes']['col_names'].values)

In [64]:
atuacao.head(3)

,id_lattes,id_instituicao,nome_instituicao,ano_inicio,ano_fim,tipo_vinculo,enquadramento_fucional,enquadramento_fucional_outro,carga_horaria_sem,dedicacao_exclusiva_flag
0,0041178224320635,010600000009,Universidade Federal de Goiás,2000,NaN,SERVIDOR_PUBLICO,LIVRE,Professor Adjunto III,0,SIM
1,0041178224320635,010600000009,Universidade Federal de Goiás,2000,2000,SERVIDOR_PUBLICO,LIVRE,Professor Adjunto II,0,SIM
2,0041178224320635,010600000009,Universidade Federal de Goiás,1997,1999,SERVIDOR_PUBLICO,LIVRE,Profesor Adjunto I,0,SIM


In [65]:
atuacao['tipo_vinculo'].unique()

array(['SERVIDOR_PUBLICO', 'OUTRO', 'LIVRE', 'CELETISTA',
       'PROFESSOR_VISITANTE', 'COLABORADOR', nan, 'BOLSISTA_RECEM_DOUTOR'],
      dtype=object)

In [66]:
atuacao['enquadramento_fucional'].unique()


array(['LIVRE', 'PROFESSOR_TITULAR', 'OUTRO', nan], dtype=object)

A princípio, iremos categorizar as classes de acordo com a variável enquadramento_funcional. Para aqueles que aparecem como "PROFESSOR_TITULAR" serão considerados ==1 (permanece na academia), os outros serão 0, não permaneceram na academia.

In [67]:
atuacao = atuacao[atuacao['enquadramento_fucional'] == 'PROFESSOR_TITULAR']
print(len(atuacao))
print(atuacao['id_lattes'].nunique())

105725
57874


Deixando o banco unitário para o id

In [68]:
atuacao = atuacao.drop_duplicates(subset='id_lattes', keep='first')
print(len(atuacao))

57874


In [69]:
atuacao = atuacao[['id_lattes']]

Adicionando a variável classe

In [71]:
atuacao['classe'] = 1

In [72]:
atuacao.head()

,id_lattes,classe
4,0041178224320635,1
169,0080556850353422,1
439,0023577321959351,1
506,0031760919656414,1
548,0013890432793373,1


Adicionando ao df principal:

In [73]:
docs = pd.merge(docs, atuacao, on='id_lattes', how='left')
docs.head()

,id_lattes,nome_completo,uf_nascimento,pais_nascimento,nacionalidade,grande_area,n_papers,n_orientacoes,n_posdoc,classe
0,0000039015885890,Tárcio de Almeida Vieira,BA,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,9.0,NaN,4.0,NaN
1,0000104116232000,Andre Zuber,PR,Brasil,B,ENGENHARIAS,8.0,9.0,1.0,NaN
2,0000111305956227,Tecia Vieira Carvalho,PI,Brasil,B,CIENCIAS_EXATAS_E_DA_TERRA,17.0,41.0,1.0,NaN
3,0000174385402367,Juliana Vieira de Freitas,SP,Brasil,B,ENGENHARIAS,5.0,NaN,NaN,NaN
4,0000183606503103,Luiza Cristina Lacerda,GO,Brasil,B,CIENCIAS_BIOLOGICAS,4.0,9.0,NaN,NaN


Apagando os dfs

In [74]:
del atuacao

# Salvando os dados em um único dataframe

In [75]:
docs.to_csv('out/docs.csv', index=False)